In [1]:
import torch
from collections import OrderedDict
from torchsig.models.iq_models.efficientnet.efficientnet import efficientnet_b4

In [2]:
# model_checkpoint = torch.load("checkpoints/v15.pt")
# num_classes = len(model_checkpoint["mdl.classifier.bias"])


In [3]:
checkpoint = "checkpoint-v17"


In [4]:
model_checkpoint = torch.load(f"checkpoints/{checkpoint}.ckpt")
model_weights = model_checkpoint["state_dict"]
model_weights = OrderedDict((k.removeprefix("mdl."),v) for k,v in model_weights.items())
torch.save(model_weights, f"weights/{checkpoint}_torchserve.pt")
num_classes = len(model_weights["classifier.bias"])

In [5]:
state_dict = torch.load(f"weights/{checkpoint}_torchserve.pt")
num_classes = len(state_dict["classifier.bias"])

model = efficientnet_b4(
    num_classes=num_classes
)

model.load_state_dict(state_dict)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

EfficientNet(
  (conv_stem): Conv1d(2, 48, kernel_size=(3,), stride=(2,), padding=(1,))
  (bn1): GBN(
    (bn): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv1d(48, 48, kernel_size=(3,), stride=(1,), padding=(1,), groups=48)
        (bn1): GBN(
          (bn): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv1d(48, 12, kernel_size=(1,), stride=(1,))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv1d(12, 48, kernel_size=(1,), stride=(1,))
        )
        (conv_pw): Conv1d(48, 24, kernel_size=(1,), stride=(1,))
        (bn2): GBN(
          (bn): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (act2): Identity()
      )


In [6]:
x = torch.randn(1,2,1024).to(device)
model(x)

tensor([[-1.2008,  0.9120,  2.9376,  1.9691, -0.3594,  2.9860]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

# JIT

In [7]:
# model_scripted = torch.jit.script(model) # Export to TorchScript
# model_scripted.save('weights/checkpoint-v15_torchscript.pt') # Save

# or 
#x = torch.randn(1,2,4096).to(device)
model.eval()
print(model(x))
jit_net = torch.jit.trace(model, x)
jit_net.save(f'weights/{checkpoint}_torchscript.pt') # Save

tensor([[-1.2008,  0.9120,  2.9376,  1.9691, -0.3594,  2.9860]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [8]:

model_jit = torch.jit.load(f'weights/{checkpoint}_torchscript.pt')
#model.eval()
model_jit(x)

tensor([[-1.2008,  0.9120,  2.9376,  1.9691, -0.3594,  2.9860]],
       device='cuda:0', grad_fn=<AddmmBackward0>)